In [ ]:
import os
from tqdm import tqdm
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
import cv2
import re
import shutil
from pathlib import Path
from sort_alpha import sorted_alpha
import random
from scipy.special import kl_div
from scipy.spatial import distance

In [ ]:
def shoulder_size(data):
    right_shoulder = [data['x'].loc[2], data['y'].loc[2]]
    left_shoulder = [data['x'].loc[5], data['y'].loc[5]]
    if right_shoulder[0] ==0:
        length = 0
    elif right_shoulder[0]>0 and left_shoulder[0]>0:
        length = distance.euclidean(right_shoulder, left_shoulder)
    else:
        length=0   
    return(length)

In [ ]:
a==2

In [ ]:
def torso_size(data):
    neck = [data['x'].loc[1], data['y'].loc[1]]
    if neck ==0:
        torso=0
        return(torso)
    hip1 = [data['x'].loc[8], data['y'].loc[8]]
    hip2 = [data['x'].loc[11], data['y'].loc[11]]
    if hip1[0]>0 and hip2[0]>0:
        probabilities = [data['p'].loc[8], data['p'].loc[11]]
        max_p = np.argmax(probabilities)
        if max_p==0:
            torso = distance.euclidean(hip1, neck)
        else:
            torso = distance.euclidean(hip2, neck)
    elif hip1[0]==0 and hip2[0]>0:
        torso = distance.euclidean(hip2, neck)
    elif hip2[0]==0 and hip1[0]>0:
        torso = distance.euclidean(hip1, neck)
    else:
        torso = 0
    return(torso)

In [ ]:
path_openpose = Path('/Users/andreibirladeanu/Documents/Data/tracking/skeletons/meal_final_tracking')
folders = sorted_alpha(path_openpose)

In [ ]:
skeleton1 = []
skeleton2 = []
for folder in folders:
    if len(sorted_alpha(os.path.join(path_openpose,folder)))<2:
        continue
    skeleton1_size = []
    skeleton2_size = []
    for skeleton in enumerate(sorted_alpha(os.path.join(path_openpose,folder))):
        if skeleton[0]==0:
            for coord in sorted_alpha(os.path.join(path_openpose,folder,skeleton[1])):
                data = pd.read_csv(os.path.join(path_openpose,folder,skeleton[1],coord))
                shoulder = shoulder_size(data)
                torso = torso_size(data)
                if size!=0 and torso!=0:
                    skeleton1_size.append(np.average([shoulder, torso], weights=[6./10 ,4./10]))
        if skeleton[0]==1:
            for coord in sorted_alpha(os.path.join(path_openpose,folder,skeleton[1])):
                data = pd.read_csv(os.path.join(path_openpose,folder,skeleton[1],coord))
                shoulder = shoulder_size(data)
                torso = torso_size(data)
                if size!=0 and torso!=0:
                    skeleton2_size.append(np.average([shoulder, torso], weights=[6./10 ,4./10]))
    if np.mean(np.array(skeleton1_size))> np.mean(np.array(skeleton2_size)):
        print("skel1",np.mean(np.array(skeleton1_size)), np.mean(np.array(skeleton2_size)), folder)
    else:
        print("skel2",np.mean(np.array(skeleton1_size)), np.mean(np.array(skeleton2_size)), folder)
    skeleton1.append(np.mean(np.array(skeleton1_size)))
    skeleton2.append(np.mean(np.array(skeleton2_size)))

In [ ]:
print('a')

In [ ]:
humans = pd.read_csv('/Users/andreibirladeanu/Documents/Code/PhD_work/openpose_tf/tracking_stats.csv')

In [ ]:
humans['size1'] = skeleton1
humans['size2'] = skeleton2

In [ ]:
humans.to_csv('humans_shoulders.csv')

In [ ]:
humans.to_csv('humans_shoulders_torso.csv')